In [35]:
#Поместим все импорты в начало ноутбука
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import ast
import math

from pyspark.sql.functions import explode, col, split, when, regexp_replace
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.ml.recommendation import ALS

from pyspark.sql import SparkSession, Row
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [2]:
#загрузим данные из 4 части
df = pd.read_csv('output_file.csv')
df = df[df['error'].isna()]
df = df.sample(frac=0.1, random_state=42).copy() #когда тестировал работал с уменьшенным датасатом для скорости.
df.drop(columns=['error'], inplace=True)
df.drop(columns=['picture'], inplace=True)
df.head(3)

,UserID,GamesOwned,name,level,badge_total,current_xp,needed_xp,friends
6257,76561199088423335,<Response [200]>,broken_wings,3.0,2.0,330.0,0.0,"['76561198098865839', '76561198109782826', '76..."
5624,76561198013613261,<Response [200]>,w3hr,11.0,5.0,1339.0,0.0,"['76561197960951610', '76561197961116309', '76..."
7385,76561197963496685,<Response [200]>,Ethan,0.0,0.0,0.0,0.0,Error: 'friendslist'


In [3]:
#Трансформация данных 1 этап
friends = []
for i in range(0,df.shape[0]):
    temp = df.iloc[i]['friends']
    if (type(temp) != type(1)):
        lst = (temp.split(","))
    friends.append(len(lst)-1)
df['friends'] = friends

friends = []
for i in range(0,df.shape[0]):
    temp = df.iloc[i]['GamesOwned']
    if (type(temp) != type(1)):
        lst = (temp.split(","))
    friends.append(len(lst)-1)
df['GamesOwnedNum'] = friends

df_new = pd.DataFrame(columns=[
    'UserID', 'app_ids', 'playtime', 'games_owned_num', 'name', 
    'level', 'badge_total', 'current_xp', 'needed_xp', 'friends'
])
for i in range(df.shape[0]):
    val = df.iloc[i]['GamesOwned']
    if (val[0] != '['):
        games = []
    else:    
        games = val.strip("[]").split("}, {")
    temp_lst = []
    temp_lst1 = []
    for el in games:
        if (el[0] != '{'): #Фиксим структуру в крайних случаях
            el = "{" + el
        if (el[-1] != '}'):
            el = el + '}'
        el = ast.literal_eval(el)
        temp_lst.append((el['appid']))
        temp_lst1.append((el['playtime_forever']))
    newrow = [
                str(df.iloc[i]['UserID']),     
                temp_lst,            
                temp_lst1,            
                df.iloc[i]['GamesOwnedNum'],       
                df.iloc[i]['name'],                 
                df.iloc[i]['level'],           
                df.iloc[i]['badge_total'],        
                df.iloc[i]['current_xp'],         
                df.iloc[i]['needed_xp'],        
                df.iloc[i]['friends']            
            ]
            
    df_new.loc[len(df_new)] = newrow

existing_spark = SparkSession.getActiveSession()
if existing_spark is not None:
    existing_spark.stop()
    
spark = SparkSession.builder \
    .appName("GameRecommendationALS") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.num.executors", "10") \
    .config("spark.eventLog.gcMetrics.youngGenerationGarbageCollectors", "G1GC") \
    .config("spark.eventLog.gcMetrics.oldGenerationGarbageCollectors", "G1GC").getOrCreate()


df_new = df_new.explode(['app_ids', 'playtime'])

df_new = df_new.dropna()

df_new = df_new.reset_index(drop=True)

df_new.head(5)

24/10/17 14:46:01 WARN Utils: Your hostname, Savvas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.6 instead (on interface en0)
24/10/17 14:46:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/17 14:46:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


,UserID,app_ids,playtime,games_owned_num,name,level,badge_total,current_xp,needed_xp,friends
0,76561198892667092,550,77,294,Lair86,8.0,5.0,872.0,0.0,0
1,76561198892667092,22380,24,294,Lair86,8.0,5.0,872.0,0.0,0
2,76561198892667092,42700,72,294,Lair86,8.0,5.0,872.0,0.0,0
3,76561198892667092,42710,10,294,Lair86,8.0,5.0,872.0,0.0,0
4,76561198892667092,105600,5002,294,Lair86,8.0,5.0,872.0,0.0,0


In [ ]:
#пустили SparkSession и подготоврили dataset для работы.

In [33]:
df = spark.createDataFrame(df_new)

user_id_mapping = df.select("UserID").distinct().withColumn("user_int_id", F.monotonically_increasing_id())
df = df.join(user_id_mapping, on="UserID", how="inner")



df_als = df.select(
    col("user_int_id"), 
    col("app_ids").alias("app_ids"), 
    col("playtime").alias("playtime")
)

df_als.show(5)


+-----------+-------+--------+
|user_int_id|app_ids|playtime|
+-----------+-------+--------+
|          0| 485610|       0|
|          0| 435790|       7|
|          0|1226470|     424|
|          0|1174180|    2579|
|          0| 950100|       0|
+-----------+-------+--------+
only showing top 5 rows



In [24]:
def test_param_grid(predictions):
    #упрощённая метрика для теста параметров модели (из урока 4)
    predictions = predictions.withColumn('pred_label', F.when(F.col('prediction') > 0, 1).otherwise(0)) #если предсказание по времение >0
    predictions = predictions.withColumn('true_label', F.when(F.col('playtime') > 0, 1).otherwise(0)) #если пользователь играл
    
    y_test = np.array(predictions.select('true_label').collect()).flatten() #адаптируем результат к нашей метрике
    y_pred_round = np.array(predictions.select('pred_label').collect()).flatten()
    
    confusion_matrix = np.zeros((2, 2), dtype=int)
    
    for true_label, pred_label in zip(y_test, y_pred_round):
        confusion_matrix[true_label, pred_label] += 1
    
    i = 1
    TP = confusion_matrix[i, i] 
    FN = confusion_matrix[i, :].sum() - TP 
    FP = confusion_matrix[:, i].sum() - TP 
    TN = confusion_matrix.sum() - (TP + FN + FP)
    
    return (2*(TP)/(TP+FP)*(TP)/(TP+FN))/((TP)/(TP+FP)+(TP)/(TP+FN))

In [32]:
#Разделим датасет на train и test
train_df, test_df = df_als.randomSplit([0.8, 0.2], seed=42)

als = ALS(
    userCol="user_int_id", 
    itemCol="app_ids", 
    ratingCol="playtime", 
    implicitPrefs=True, 
    coldStartStrategy="drop" #важно иначе будет НаН из за невозможности адекватно сделать предсказание
)

#найдём оптимальные параметры модели
maxIter = [5, 10, 20]
regParam = [0.01, 0.1, 0.2]
rank = [10, 20, 50]

lstf1 = []
#я сделал подбор параметров в ручную
for el1 in maxIter:
    for el2 in regParam:
        for el3 in rank:
            als.setMaxIter(el1)
            als.setRegParam(el2)
            als.setRank(el3)
            als_model = als.fit(train_df)
            predictions = als_model.transform(test_df)
            lstf1.append((test_param_grid(predictions), el1, el2, el3))
            print(lstf1[-1])

max_fit_tuple = max(lstf1, key=lambda x: x[0])
print("Лучшие параметры:", max_fit_tuple)

als.setMaxIter(max_fit_tuple[1])
als.setRegParam(max_fit_tuple[2])
als.setRank(max_fit_tuple[3])

#Фитим лучшую модель
als_model = als.fit(train_df)
predictions = als_model.transform(test_df)


(0.7533502354219486, 5, 0.01, 10)


(0.751945055534689, 5, 0.01, 20)


(0.6807161725858526, 5, 0.01, 50)


(0.7542084064855933, 5, 0.1, 10)


(0.7542173820200726, 5, 0.1, 20)


...

(0.7591729913635175, 20, 0.1, 20)


(0.7179027559937262, 20, 0.1, 50)


(0.7647148754013149, 20, 0.2, 10)


(0.7626244202407629, 20, 0.2, 20)


(0.6174667182234989, 20, 0.2, 50)
Лучшие параметры: (0.7647148754013149, 20, 0.2, 10)


In [30]:
predictions.head(10)

[Row(user_int_id=1, app_ids=310950, playtime=569, prediction=0.9223475456237793),
 Row(user_int_id=7, app_ids=945360, playtime=5953, prediction=1.0353220701217651),
 Row(user_int_id=11, app_ids=945360, playtime=2436, prediction=1.0013173818588257),
 Row(user_int_id=22, app_ids=945360, playtime=14232, prediction=1.0074678659439087),
 Row(user_int_id=28, app_ids=945360, playtime=727, prediction=0.9778640270233154),
 Row(user_int_id=32, app_ids=222730, playtime=194, prediction=0.6599942445755005),
 Row(user_int_id=40, app_ids=847370, playtime=26, prediction=-0.013680696487426758),
 Row(user_int_id=40, app_ids=945360, playtime=2863, prediction=1.0137982368469238),
 Row(user_int_id=41, app_ids=945360, playtime=648, prediction=0.9855535626411438),
 Row(user_int_id=55, app_ids=945360, playtime=3526, prediction=1.0146288871765137)]

In [31]:
#Протестим результаты той же метрикой, что и в прошлый раз

predictions = predictions.withColumn('pred_label', F.when(F.col('prediction') > 0, 1).otherwise(0)) #если предсказание по времение >0
predictions = predictions.withColumn('true_label', F.when(F.col('playtime') > 0, 1).otherwise(0)) #если пользователь играл

y_test = np.array(predictions.select('true_label').collect()).flatten() #адаптируем результат к нашей метрике
y_pred_round = np.array(predictions.select('pred_label').collect()).flatten()

confusion_matrix = np.zeros((2, 2), dtype=int)

for true_label, pred_label in zip(y_test, y_pred_round):
    confusion_matrix[true_label, pred_label] += 1

i = 1
TP = confusion_matrix[i, i] 
FN = confusion_matrix[i, :].sum() - TP 
FP = confusion_matrix[:, i].sum() - TP 
TN = confusion_matrix.sum() - (TP + FN + FP)

TPF += TP
FNF += FN
FPF += FP
TNF += TN

print(f"\nTarget = {i}:")
print(f"True Positives (TP): {TP}")
print(f"False Negatives (FN): {FN}")
print(f"False Positives (FP): {FP}")
print(f"True Negatives (TN): {TN}")

print(f'Accuracy: {(TP+TN)/(TP+TN+FP+FN)}')
print(f'Precision: {(TP)/(TP+FP)}')
print(f'Recall: {(TP)/(TP+FN)}') 
print(f'F1 Score: {(2*(TP)/(TP+FP)*(TP)/(TP+FN))/((TP)/(TP+FP)+(TP)/(TP+FN))}')
print(f'Matthews Correlation Coefficient: {((TP*TN)-(FP*FN))/(math.sqrt((TP+FN)*(TP+FP)*(TN+FN)*(TN+FP)))}') #так как на лицо дизбаланс классов


Target = 1:
True Positives (TP): 7503
False Negatives (FN): 1045
False Positives (FP): 3572
True Negatives (TN): 2528
Accuracy: 0.684803386127799
Precision: 0.6774717832957111
Recall: 0.877749181094993
F1 Score: 0.7647148754013149
Matthews Correlation Coefficient: 0.33539181653907785


In [ ]:
"""
Заключение
В сравнение с предыдущей части следующие моменты были исправленны в соответствие с feedback-ом

1.Все импорты перенесенны в начало ноутбука

2.Приведение системы оценки качества работы модели к общему знаменателю с оценкой из предыдущей части

3.Улучшения байзлана путём работы с усложнёнными классическими рекомендательными моделями

4.Реализован подбор параметров с поиском наиболее эффектиконо

Результаты
Мы создали более сложную модель, которая научилась определять заинтересуюет ли пользователя та или иная игра (на базе playtime). В отличие от
4 части наша модель работает куда быстрее и требует меньше данных, что критично для нашего сервиса и его дальнейшего маштабирования.
У нас очень хороший recall, а относительно низний precision не являеться проблемой, так как мы реккомендуем сразу несколько игр!
"""